In [1]:
import cv2
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

In [2]:
# Defina uma variável global para armazenar o modelo treinado
global model
model = None

# Obtenha o diretório atual de trabalho
script_dir = os.getcwd()

# Define o diretório onde suas imagens estão localizadas
data_dir = os.path.join(script_dir, 'Base_de_Dados')

In [3]:
def preprocess_image(img_path):
    # Carregue a imagem original
    imagem = cv2.imread(img_path)

    # Verifique se a imagem foi carregada com sucesso
    if imagem is None:
        print(f"Erro ao carregar a imagem: {img_path}")
        return None  # Retorne None para indicar que o processamento falhou

    # Redimensionando a imagem
    imagem_redimensionada = cv2.resize(imagem, (224, 224))

    # Aumento de contraste
    imagem_em_escala_de_cinza = cv2.cvtColor(imagem_redimensionada, cv2.COLOR_BGR2GRAY)
    #imagem_em_escala_de_cinza = cv2.equalizeHist(imagem_em_escala_de_cinza)

    # Retorne a imagem processada
    return imagem_em_escala_de_cinza  # Mantenha a matriz 2D em escala de cinza com aumento de contraste

In [4]:
# Use ImageDataGenerator para carregar os dados
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255)  # Normalização

# Carregue os dados em um array numpy
image_data = []
image_labels = []

for subdir in os.listdir(data_dir):
    sub_dir_path = os.path.join(data_dir, subdir)
    if os.path.isdir(sub_dir_path):
        for img_file in os.listdir(sub_dir_path):
            img_path = os.path.join(sub_dir_path, img_file)
            image_data.append(img_path)
            image_labels.append(subdir)

image_data = np.array(image_data)
image_labels = np.array(image_labels)
labels = np.unique(image_labels)

# Defina o número de folds
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Variáveis para armazenar resultados
all_train_histories = []
all_val_histories = []
confusion_matrices = []

# Defina o valor de batch_size
batch_size = 32  # Você pode ajustar esse valor conforme necessário

In [5]:
def treino(nome_do_modelo):
    global model  # Use a variável global

    # Variável para armazenar informações de cada fold
    fold_info = []

    # Loop através dos folds
    for i, (train_index, val_index) in enumerate(kf.split(image_data), 1):
        train_data = image_data[train_index]
        train_labels = image_labels[train_index]
        val_data = image_data[val_index]
        val_labels = image_labels[val_index]

        # Pré-processar as imagens de treinamento e validação
        train_data_processed = [preprocess_image(img_path) for img_path in train_data]
        val_data_processed = [preprocess_image(img_path) for img_path in val_data]

        # Remova entradas None (imagens que não puderam ser carregadas)
        train_data_processed = [img for img in train_data_processed if img is not None]
        val_data_processed = [img for img in val_data_processed if img is not None]

        # Converter as imagens em listas de matrizes 1D
        train_data_processed = [img.flatten() for img in train_data_processed]
        val_data_processed = [img.flatten() for img in val_data_processed]

        # Converta as listas em arrays numpy
        train_data_processed = np.array(train_data_processed)
        val_data_processed = np.array(val_data_processed)

        train_df = pd.DataFrame({'filename': train_data, 'class': train_labels})
        val_df = pd.DataFrame({'filename': val_data, 'class': val_labels})

        train_generator = datagen.flow_from_dataframe(
            dataframe=train_df,
            x_col='filename',
            y_col='class',
            target_size=(224, 224),
            batch_size=batch_size,
            class_mode='categorical',
            shuffle=True
        )

        val_generator = datagen.flow_from_dataframe(
            dataframe=val_df,
            x_col='filename',
            y_col='class',
            target_size=(224, 224),
            batch_size=batch_size,
            class_mode='categorical',
            shuffle=False
        )

        # Crie e compile o modelo (certifique-se de usar o número correto de classes)
        num_classes = len(np.unique(train_labels))
        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))
        model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))
        model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(64, activation='relu'))
        model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))  # Use o número correto de unidades

        model.compile(optimizer='rmsprop',
                    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                    metrics=['accuracy'])

        # Treine o modelo por 5 época (ou o número de épocas desejado)
        history = model.fit(train_generator, epochs=5, validation_data=val_generator)

        # Avalie o modelo no conjunto de teste (val_generator)
        y_true = val_generator.classes  # Rótulos reais
        y_pred_probs = model.predict(val_generator)  # Probabilidades das previsões
        y_pred = np.argmax(y_pred_probs, axis=1)  # Rótulos previstos

        # Calcule a matriz de confusão
        cm = confusion_matrix(y_true, y_pred)

        # Calcula a precisão para cada classe
        num_classes = cm.shape[0]
        precisao_por_classe = []
        for classe in range(num_classes):
            TP = cm[classe, classe]  # Verdadeiros Positivos
            FP = np.sum(cm[:, classe]) - TP  # Falsos Positivos
            precision = TP / (TP + FP)
            precisao_por_classe.append(precision)

        # Armazena informações do fold em fold_info
        fold_info.append({
            'Fold': i,
            'Loss': history.history['loss'],
            'Accuracy': history.history['accuracy'],
            'Validation Loss': history.history['val_loss'],
            'Validation Accuracy': history.history['val_accuracy'],
            'Confusion Matrix': cm,
            'Precision per Class': precisao_por_classe
        })

    # Salve o modelo treinado com o nome especificado
    model.save(nome_do_modelo)

    # Retorne o modelo e as informações de cada fold
    return model, fold_info


In [6]:
def calcular_matriz_confusao_e_precisao(model, nome_do_modelo, labels, confusion_matrices):
    # Crie a matriz de confusão geral
    overall_confusion_matrix = np.sum(confusion_matrices, axis=0)

    # Calcula a precisão para cada classe
    num_classes = overall_confusion_matrix.shape[0]
    precisao_por_classe = []

    for classe in range(num_classes):
        TP = overall_confusion_matrix[classe, classe]  # Verdadeiros Positivos
        FP = np.sum(overall_confusion_matrix[:, classe]) - TP  # Falsos Positivos
        precision = TP / (TP + FP)
        precisao_por_classe.append(precision)

    return {
        'Matriz de Confusão Geral': overall_confusion_matrix,
        'Precisão por Classe': precisao_por_classe
    }

In [28]:
# Função para exibir informações de cada fold
def exibir_informacoes_fold(fold_info, labels):
    all_accuracy = []
    all_val_accuracy = []
    all_precision = []
    all_confusion_matrices = []

    for fold_data in fold_info:
        fold = fold_data['Fold']
        loss = fold_data['Loss']
        accuracy = fold_data['Accuracy']
        val_loss = fold_data['Validation Loss']
        val_accuracy = fold_data['Validation Accuracy']
        confusion_matrix_fold = fold_data['Confusion Matrix']
        precisao_por_classe_fold = fold_data['Precision per Class']

        print(f'Fold {fold}:')
        print(f'Loss: {loss}')
        print(f'Accuracy: {accuracy}')
        print(f'Validation Loss: {val_loss}')
        print(f'Validation Accuracy: {val_accuracy}')

        print("Matriz de Confusão:")
        print(confusion_matrix_fold)

        print("\nPrecisão por Classe:")
        for classe, precision in enumerate(precisao_por_classe_fold):
            classe_nome = labels[classe]
            print(f"Classe '{classe_nome}': Precisão = '{precision:.4f}'")

        all_accuracy.append(accuracy[-1])
        all_val_accuracy.append(val_accuracy[-1])
        all_precision.append(precisao_por_classe_fold)
        all_confusion_matrices.append(confusion_matrix_fold)

    num_folds = len(fold_info)

    # Calcular médias
    average_accuracy = np.mean(all_accuracy)
    average_val_accuracy = np.mean(all_val_accuracy)
    average_precision = np.mean(all_precision, axis=0)

    print(f'\nAcurácia Média entre os {num_folds} folds: {average_accuracy:.4f}')
    print(f'Acurácia de Validação Média entre os {num_folds} folds: {average_val_accuracy:.4f}')

    # Calcular a matriz de confusão geral (somatório das matrizes dos folds)
    confusion_matrix_general = np.sum(all_confusion_matrices, axis=0)

    print("\nMatriz de Confusão Geral (Somatório das Matrizes dos Folds):")
    print(confusion_matrix_general)

    print("\nPrecisão Média por Classe:")
    for classe, precision in enumerate(average_precision):
        classe_nome = labels[classe]
        print(f"Classe '{classe_nome}': Precisão Média = '{precision:.4f}'")


In [8]:
# Função para carregar o modelo treinado com um nome especificado ou treiná-lo se necessário
def carregar_ou_treinar_modelo(nome_do_modelo):
    global model
    if model is None:
        if os.path.exists(nome_do_modelo):
            model = tf.keras.models.load_model(nome_do_modelo)
        else:
            model = treino(nome_do_modelo)
    return model

In [9]:
def preprocess_image_for_prediction(img_path):
    # Carregue a imagem original
    imagem = cv2.imread(img_path)

    # Verifique se a imagem foi carregada com sucesso
    if imagem is None:
        print(f"Erro ao carregar a imagem: {img_path}")
        return None  # Retorne None para indicar que o processamento falhou

    # Redimensionando a imagem
    imagem_redimensionada = cv2.resize(imagem, (224, 224))

    # Aumento de contraste
    imagem_em_escala_de_cinza = cv2.cvtColor(imagem_redimensionada, cv2.COLOR_BGR2GRAY)
    #imagem_em_escala_de_cinza = cv2.equalizeHist(imagem_em_escala_de_cinza)

    # Expanda as dimensões para corresponder à forma esperada do modelo (1, 224, 224, 3)
    imagem_processada = np.expand_dims(imagem_em_escala_de_cinza, axis=0)
    imagem_processada = np.stack((imagem_processada,) * 3, axis=-1)  # Replica o canal em todos os três canais
    imagem_processada = np.expand_dims(imagem_processada, axis=-1)
    
    # Normalizar a imagem
    imagem_processada = imagem_processada / 255.0

    return imagem_processada

In [10]:
# Função para fazer a classificação da imagem
def classificar_imagem(imagem_path, nome_do_modelo):
    model = carregar_ou_treinar_modelo(nome_do_modelo)  # Carrega ou treina o modelo
    # Pré-processamento da imagem
    imagem_processada = preprocess_image_for_prediction(imagem_path)

    if imagem_processada is None:
        return "Falha no pré-processamento", None

    # Faça a previsão usando o modelo
    previsao = model.predict(imagem_processada)

    # Obtenha a classe prevista com a maior probabilidade
    classe_prevista = np.argmax(previsao)

    # Imprima as classes previstas e suas probabilidades
    print(f"\n\nResultado da classificação para a imagem: {imagem_path} usando o modelo: {nome_do_modelo}")
    print("\nClassificação das 3 classes mais prováveis:")
    top_classes_indices = np.argsort(previsao[0])[::-1][:3]
    top_classes_names = [labels[i] for i in top_classes_indices]
    top_classes_probs = [previsao[0][i] for i in top_classes_indices]
    for i, (classe, probabilidade) in enumerate(zip(top_classes_names, top_classes_probs), 1):
        print(f"{i}. Classe: {classe}, Probabilidade: {probabilidade:.4f}")

    # Imprima todas as classes e suas probabilidades
    print("\nProbabilidades para todas as classes:")
    for i, (classe, probabilidade) in enumerate(zip(labels, previsao[0]), 1):
        print(f"{i}. Classe: {classe}, Probabilidade: {probabilidade:.4f}")

    return labels[classe_prevista], previsao[0][classe_prevista]

In [11]:
# Especifique o nome do modelo desejado
nome_do_modelo = 'image_classification.h5'

# Carregue ou treine o modelo e obtenha as informações/estatísticas do fold
model, fold_info = carregar_ou_treinar_modelo(nome_do_modelo)

Found 5132 validated image filenames belonging to 8 classes.
Found 1283 validated image filenames belonging to 8 classes.


e:\anaconda3\lib\site-packages\keras\src\preprocessing\image.py:1137: UserWarning: Found 1 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


Epoch 1/5
161/161 [==============================] - 312s 2s/step - loss: 1.9066 - accuracy: 0.4084 - val_loss: 0.8769 - val_accuracy: 0.6999
Epoch 2/5
161/161 [==============================] - 289s 2s/step - loss: 0.7269 - accuracy: 0.7399 - val_loss: 0.5768 - val_accuracy: 0.8059
Epoch 3/5
161/161 [==============================] - 289s 2s/step - loss: 0.3738 - accuracy: 0.8784 - val_loss: 0.5348 - val_accuracy: 0.8153
Epoch 4/5
161/161 [==============================] - 276s 2s/step - loss: 0.1845 - accuracy: 0.9415 - val_loss: 0.7273 - val_accuracy: 0.7958
Epoch 5/5
41/41 [==============================] - 25s 580ms/step
Found 5132 validated image filenames belonging to 8 classes.


e:\anaconda3\lib\site-packages\keras\src\preprocessing\image.py:1137: UserWarning: Found 1 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


Found 1283 validated image filenames belonging to 8 classes.
Epoch 1/5
161/161 [==============================] - 293s 2s/step - loss: 1.8215 - accuracy: 0.4452 - val_loss: 1.2153 - val_accuracy: 0.5370
Epoch 2/5
161/161 [==============================] - 288s 2s/step - loss: 0.7266 - accuracy: 0.7504 - val_loss: 0.6766 - val_accuracy: 0.7553
Epoch 3/5
161/161 [==============================] - 285s 2s/step - loss: 0.3998 - accuracy: 0.8714 - val_loss: 0.7896 - val_accuracy: 0.7381
Epoch 4/5
161/161 [==============================] - 292s 2s/step - loss: 0.2076 - accuracy: 0.9396 - val_loss: 0.6431 - val_accuracy: 0.8051
Epoch 5/5
41/41 [==============================] - 30s 706ms/step
Found 5132 validated image filenames belonging to 8 classes.


...

e:\anaconda3\lib\site-packages\keras\src\preprocessing\image.py:1137: UserWarning: Found 1 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


Found 1283 validated image filenames belonging to 8 classes.
Epoch 1/5
161/161 [==============================] - 232s 1s/step - loss: 1.5534 - accuracy: 0.4550 - val_loss: 0.9372 - val_accuracy: 0.6804
Epoch 2/5
161/161 [==============================] - 228s 1s/step - loss: 0.6678 - accuracy: 0.7687 - val_loss: 0.7177 - val_accuracy: 0.7623
Epoch 3/5
161/161 [==============================] - 221s 1s/step - loss: 0.3270 - accuracy: 0.8899 - val_loss: 0.6047 - val_accuracy: 0.8012
Epoch 4/5
161/161 [==============================] - 222s 1s/step - loss: 0.1556 - accuracy: 0.9509 - val_loss: 0.7821 - val_accuracy: 0.8028
Epoch 5/5
41/41 [==============================] - 18s 442ms/step
Found 5132 validated image filenames belonging to 8 classes.


e:\anaconda3\lib\site-packages\keras\src\preprocessing\image.py:1137: UserWarning: Found 1 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


Found 1283 validated image filenames belonging to 8 classes.
Epoch 1/5
161/161 [==============================] - 234s 1s/step - loss: 1.7402 - accuracy: 0.3971 - val_loss: 1.1633 - val_accuracy: 0.5939
Epoch 2/5
161/161 [==============================] - 236s 1s/step - loss: 0.7832 - accuracy: 0.7210 - val_loss: 0.6570 - val_accuracy: 0.7724
Epoch 3/5
161/161 [==============================] - 227s 1s/step - loss: 0.3847 - accuracy: 0.8704 - val_loss: 0.6229 - val_accuracy: 0.7958
Epoch 4/5
161/161 [==============================] - 231s 1s/step - loss: 0.1789 - accuracy: 0.9478 - val_loss: 0.8029 - val_accuracy: 0.7981
Epoch 5/5
41/41 [==============================] - 19s 450ms/step


e:\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
# Insira o caminho da imagem que você deseja classificar
imagem_path = os.path.join(script_dir, '73.jpg')

# Em seguida, você pode realizar a classificação da imagem conforme anteriormente
classe_prevista = classificar_imagem(imagem_path, model)
print(f'\nO nome da classe prevista é: {classe_prevista}','\n')

1/1 [==============================] - 0s 118ms/step


Resultado da classificação para a imagem: e:\zzzzzanime\73.jpg usando o modelo: <keras.src.engine.sequential.Sequential object at 0x000001478EC811B0>

Classificação das 3 classes mais prováveis:
1. Classe: Pica_Pau, Probabilidade: 0.4996
2. Classe: Bungo_Stray_Dogs, Probabilidade: 0.4409
3. Classe: Kick_Buttowski, Probabilidade: 0.0316

Probabilidades para todas as classes:
1. Classe: Apenas_Um_Show, Probabilidade: 0.0016
2. Classe: Bob_Esponja, Probabilidade: 0.0000
3. Classe: Bungo_Stray_Dogs, Probabilidade: 0.4409
4. Classe: Kick_Buttowski, Probabilidade: 0.0316
5. Classe: Looney_Tunes, Probabilidade: 0.0261
6. Classe: Madeline, Probabilidade: 0.0001
7. Classe: Padrinhos_Magicos, Probabilidade: 0.0001
8. Classe: Pica_Pau, Probabilidade: 0.4996

O nome da classe prevista é: ('Pica_Pau', 0.49963942) 



In [29]:
# Exiba as informações/estatísticas do fold
exibir_informacoes_fold(fold_info, labels)

Fold 1:
Loss: [1.9065566062927246, 0.7269456386566162, 0.37381094694137573, 0.18453657627105713, 0.11859223991632462]
Accuracy: [0.4084177613258362, 0.7398675084114075, 0.8784099817276001, 0.9415432810783386, 0.965120792388916]
Validation Loss: [0.8768516182899475, 0.5768245458602905, 0.5348190665245056, 0.7272940278053284, 0.6471142768859863]
Validation Accuracy: [0.6999220848083496, 0.8059236407279968, 0.8152766823768616, 0.7957910895347595, 0.8020265102386475]
Matriz de Confusão:
[[575   0  27  44  41  46   1  66]
 [  1 789   0   0   1   5   0   3]
 [ 26   0 667  21  21  24   1  40]
 [ 68   1  22 492  61  36   2 118]
 [ 42   1  26  42 505  53   1 130]
 [ 20   3  15  11  62 613   2  80]
 [  1   0   1   1   1   3 797   6]
 [ 19   1  18  19  75  47   1 620]]

Precisão por Classe:
Classe 'Apenas_Um_Show': Precisão = '0.7488'
Classe 'Bob_Esponja': Precisão = '0.9937'
Classe 'Bungo_Stray_Dogs': Precisão = '0.8836'
Classe 'Kick_Buttowski': Precisão = '0.8525'
Classe 'Looney_Tunes': Precisã